#### 회귀분석
지역명, 규모, 시기, 금리, 산업생산지수, 소비자 물가지수, 부동산 정책(이슈)를 바탕으로 아파트 분양가격 예측

#### 경기 선행/후행지수인지
- 경기지표 vs 부동산 시장

#### 지역별로 그룹핑하여 PCA
-주성분 분석하여 서울지역이 지역primium이 있는지

https://datascienceschool.net/view-notebook/58269d7f52bd49879965cdc4721da42d/

In [147]:
df= pd.read_csv(
 "주택도시보증공사_전국 평균 분양가격(2019년 9월).csv", encoding='euc-kr'
)
# df["월"].unique()
df.tail()

,지역명,규모구분,연도,월,분양가격(㎡)
4075,제주,전체,2019,9,3869
4076,제주,전용면적 60㎡이하,2019,9,3804
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884
4078,제주,전용면적 85㎡초과 102㎡이하,2019,9,NaN
4079,제주,전용면적 102㎡초과,2019,9,3601


In [148]:
df.dtypes

지역명        object
규모구분       object
연도          int64
월           int64
분양가격(㎡)    object
dtype: object

In [165]:
#NAN값 삭제
df=df[df["분양가격(㎡)"].notnull()]

In [166]:
# "  " 삭제
df=df[df["분양가격(㎡)"]!="  "]

In [167]:
# ," 지우기
df["분양가격(㎡)"].replace(",","").tail()

4074    4303.0
4075    3869.0
4076    3804.0
4077    3884.0
4079    3601.0
Name: 분양가격(㎡), dtype: float64

In [168]:
#분양가격 숫자형 데이터로 바꾸기
df["분양가격(㎡)"] = pd.to_numeric(df["분양가격(㎡)"], errors='coerce')

In [169]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3726 entries, 0 to 4079
Data columns (total 5 columns):
지역명        3726 non-null object
규모구분       3726 non-null object
연도         3726 non-null int64
월          3726 non-null int64
분양가격(㎡)    3726 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 174.7+ KB


In [170]:
df.describe()

,연도,월,분양가격(㎡)
count,3726.000000,3726.000000,3726.000000
mean,2017.217660,6.420827,3201.605743
std,1.206523,3.427099,1223.855504
min,2015.000000,1.000000,1868.000000
25%,2016.000000,3.000000,2418.500000
50%,2017.000000,6.000000,2855.000000
75%,2018.000000,9.000000,3508.000000
max,2019.000000,12.000000,12728.000000


In [171]:
# 구글 스프레드시트에서 보기
import pygsheets
gc= pygsheets.authorize(client_secret="client_secret.json", no_cash=True)
s_file = gc.open("zigbang")
apt_sheet=s_file.add_worksheet("apt")
apt_sheet.set_dataframe(df,"A1", copy_index=False)

In [172]:
df_1=df.copy()

In [173]:
df_1["연도"]=df["연도"].apply(str)
df_1["월"]=df["월"].apply(str)
df_1["시기"]=df_1["연도"]+"0"+df_1["월"]
df_1.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기
4074,경남,전용면적 102㎡초과,2019,9,4303.0,201909
4075,제주,전체,2019,9,3869.0,201909
4076,제주,전용면적 60㎡이하,2019,9,3804.0,201909
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
4079,제주,전용면적 102㎡초과,2019,9,3601.0,201909


In [197]:
df_2=df_1.copy()
df_2.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기
4074,경남,전용면적 102㎡초과,2019,9,4303.0,201909
4075,제주,전체,2019,9,3869.0,201909
4076,제주,전용면적 60㎡이하,2019,9,3804.0,201909
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
4079,제주,전용면적 102㎡초과,2019,9,3601.0,201909


In [198]:
df_2= pd.get_dummies(df_2["지역명"])
df_2.tail()

,강원,경기,경남,경북,광주,대구,대전,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
4074,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4075,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4076,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4077,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4079,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [204]:
df_3=pd.concat([df_1,df_2], axis=1)
df_3.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기,강원,경기,경남,경북,...,부산,서울,세종,울산,인천,전남,전북,제주,충남,충북
4074,경남,전용면적 102㎡초과,2019,9,4303.0,201909,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4075,제주,전체,2019,9,3869.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4076,제주,전용면적 60㎡이하,2019,9,3804.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4077,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4079,제주,전용면적 102㎡초과,2019,9,3601.0,201909,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [205]:
#지금까지 한 df_1 csv로 저장하기
df_1.to_csv('df_1.csv',sep=",",index= False)
df_2.to_csv('df_2.csv',sep=",",index= False)
df_3.to_csv('df_3.csv',sep=",",index= False)

In [183]:
df= pd.read_csv(
 "df_1.csv")
df.tail()

,지역명,규모구분,연도,월,분양가격(㎡),시기
3721,경남,전용면적 102㎡초과,2019,9,4303.0,201909
3722,제주,전체,2019,9,3869.0,201909
3723,제주,전용면적 60㎡이하,2019,9,3804.0,201909
3724,제주,전용면적 60㎡초과 85㎡이하,2019,9,3884.0,201909
3725,제주,전용면적 102㎡초과,2019,9,3601.0,201909


https://data.seoul.go.kr/dataList/datasetView.do?infId=524&srvType=S&serviceKind=2

In [5]:
cofix= pd.read_csv(
 "Cofix.csv", encoding='euc-kr',header=1
)
cofix_df=cofix[cofix.columns[1:3]]
cofix_df.sort_values(["대상월"], ascending=True, inplace=True)
cofix_df=cofix_df[cofix_df["대상월"]!="2014/12"]
cofix_df.reset_index(drop=True, inplace=True)
cofix_df.tail()

,대상월,신규취급액기준 COFIX
52,2019/05,1.85
53,2019/06,1.78
54,2019/07,1.68
55,2019/08,1.52
56,2019/09,1.57


In [9]:
cpi= pd.read_csv(
 "cpi.csv", encoding='euc-kr',header=0
)
cpi_df=cpi[["기간","총지수"]]

In [17]:
#산업생산지수(index of Industrial Production)
#http://www.index.go.kr/potal/main/EachDtlPageDetail.do?idx_cd=2838
iaip= pd.read_csv(
 "iaip.csv", encoding='euc-kr', header=0)
iaip=iaip.T
iaip.columns=["전산업생산지수(원지수)","전월동기대비","광공업","건설업","서비스업","공공행정"] #나중에 뭘 쓸지 몰라서 우선 모두 표시
iaip=iaip.drop(iaip.index[0])
iaip_df=pd.DataFrame(iaip["전산업생산지수(원지수)"])
iaip_df.tail()

,전산업생산지수(원지수)
201905월,108.7
201906월,109
201907월,107.8
201908월,105.4
201909월,105.3
